In [1]:
import numpy as np
import cv2 as cv

In [2]:
np.set_printoptions(suppress=True)

In [3]:
from tpk4170.visualization import Viewer
from tpk4170.models import Ball, Grid, Axes
from tpk4170.utils.transformations import quaternion_from_matrix, unit_vector

In [8]:
viewer = Viewer()
viewer.add(Grid())
viewer.add(Axes(0.5))
cam_axes = Axes(0.5)
viewer.add(cam_axes)

Renderer(camera=PerspectiveCamera(aspect=1.5, children=(DirectionalLight(color='white', intensity=0.66, positi…

In [9]:
camera_matrix = np.array([[750.0, 0.0, 1280/2], 
              [0.0, 750.0, 1024/2], 
              [0.0, 0.0, 1.0]])
dist_coeffs = np.zeros((5,1))

In [10]:
tvec = np.array([0.5, 0.5, 2.0])
R = np.array([[1.0, 0.0, 0.0, 0.0], 
              [0.0, -1.0, 0.0, 0.0], 
              [0.0, 0.0, -1.0, 0.0],
              [0.0,0.0,0.0,1.0]])
rvec, _ = cv.Rodrigues(R[:3,:3])

In [11]:
cam_axes.position = tvec.tolist()
cam_axes.quaternion = quaternion_from_matrix(R).tolist()

In [14]:
obj_points = np.array([[0.5, 0.5, 0.0], 
                       [0.5, 1.5, 0.0], 
                       [1.5, 1.5, 0.0], 
                       [1.5, 0.5, 0.0]]) /2.0

In [15]:
for p in obj_points:
    ball = Ball()
    ball.position = p.tolist()
    viewer.add(ball)

In [17]:
img_points, _ = cv.projectPoints(obj_points, rvec, tvec, camera_matrix, dist_coeffs)
img_points

array([[[ 921.25,  605.75]],

       [[ 921.25,  418.25]],

       [[1108.75,  418.25]],

       [[1108.75,  605.75]]])

In [18]:
cv.undistortPoints(img_points, camera_matrix, dist_coeffs).reshape(-1,2)

array([[ 0.375,  0.125],
       [ 0.375, -0.125],
       [ 0.625, -0.125],
       [ 0.625,  0.125]])

In [19]:
ret, rvecs, tvecs = cv.solveP3P(obj_points[:3], img_points[:3], camera_matrix, dist_coeffs, cv.SOLVEPNP_AP3P)
print(ret)
for r, t in zip(rvecs, tvecs):
    R = np.eye(4)
    R[:3,:3] = cv.Rodrigues(r)[0]
#     print(t)
#     print(R)
#     print()
    axes = Axes(0.5)
    axes.position = t.ravel().tolist()
    axes.quaternion = quaternion_from_matrix(R).tolist()
    viewer.add(axes)

4


In [20]:
ret, rvecs, tvecs = cv.solvePnP(obj_points, img_points, camera_matrix, dist_coeffs, cv.SOLVEPNP_P3P)
rvecs, tvecs

(array([[ 3.14159265],
        [ 0.        ],
        [-0.        ]]), array([[0.5],
        [0.5],
        [2. ]]))

In [21]:
def cos_angle_between_vectors(a,b):
    a = a / np.linalg.norm(a)
    b = b / np.linalg.norm(b)
    cos_theta = np.dot(a,b)
    return cos_theta
    
def angle_between_vectors(a,b):
    return np.arctan2(np.linalg.norm(np.cross(a,b)), np.dot(a,b))

In [22]:
a, b, c = img_points[:3].reshape(-1,2)

In [23]:
b

array([921.25, 418.25])

In [24]:
obj_points

array([[0.25, 0.25, 0.  ],
       [0.25, 0.75, 0.  ],
       [0.75, 0.75, 0.  ],
       [0.75, 0.25, 0.  ]])

In [25]:
def p3p_coeffs(Rab, Rac, Rbc, Cab, Cac, Cbc):
    K1 = Rbc**2 / Rac**2
    K2 = Rbc**2 / Rab**2
    
    G4 = (K1*K2-K1-K2)**2 - 4*K1*K2*Cbc**2
    
    G3 = 4*(K1*K2-K1-K2)*K2*(1-K1)*Cab + 4*K1*Cbc*((K1*K2-K1+K2)*Cac + 2*K2*Cab*Cbc)
    
    G2 = (2*K2*(1-K1)*Cab)**2 +\
            2*(K1*K2-K1-K2)*(K1*K2+K1-K2) +\
            4*K1*((K1-K2)*Cbc**2 + K1*(1-K2)*Cac**2 - 2*(1+K1)*K2*Cab*Cac*Cbc)
    
    G1 = 4*(K1*K2+K1-K2)*K2*(1-K1)*Cab +\
            4*K1*((K1*K2-K1+K2)*Cac*Cbc + 2*K1*K2*Cab*Cac**2)
    
    G0 = (K1*K2+K1-K2)**2 - 4*K1**2*K2*Cac**2
    
    coeffs = (G4, G3, G2, G1, G0)
    
    xs = np.roots(coeffs)
    xs = np.real(xs[xs.imag < 1e-5])
    
    return xs

In [26]:
p3p_coeffs(2*np.sqrt(3), 2*np.sqrt(3), 2*np.sqrt(3), 20/32, 20/32., 20/32)

array([4.  , 1.  , 1.  , 0.25])

In [27]:
def p3p(obj_points, img_points, camera_matrix, dist_coeffs):
    A, B, C = obj_points.reshape(-1,3)[:3]
    
    hom_img_points = np.hstack((img_points[:3].reshape(-1,2), 
                                np.ones((3,1)))).T

    x, y, z = (np.linalg.inv(camera_matrix) @ hom_img_points).T
    
    Rab = np.linalg.norm(B-A)
    Rac = np.linalg.norm(C-A)
    Rbc = np.linalg.norm(C-B)
    
#     Rab = Rac = Rbc = 2 * np.sqrt(3)
    
    Cab = np.cos(angle_between_vectors(x,y))
    Cac = np.cos(angle_between_vectors(x,z))
    Cbc = np.cos(cos_angle_between_vectors(y,z))
    
    print(Cab, Cac, Cbc)
    
#     Cab = Cac = Cbc = 20./32.
    
    xs = p3p_coeffs(Rab, Rac, Rbc, Cab, Cac, Cbc)
    
    as_ = np.array([Rab / np.sqrt(x**2 - 2*x*Cab + 1) for x in xs])
    
#     bs = [a * x for a,x in zip(as_, xs)]
    
#     def compute_y(x, a, b):
#         m = 1-K1
#         p = 2*(K1*Cac-x*Cbc)
#         q = x**2-K1
#         m_ = 1
#         p_ = 2*(-x*Cbc)
#         q_ = (x**2*(1-K2)+2*x*K2*Cab-K2)
#         num = (p_*q - p*q_)
#         den = (m*q_ - m_*q)
#         if np.allclose(den, 0.0):
#             y = Cac + np.sqrt(Cac**2 + (Rac**2 - a**2)/a**2)
#             print(y)
#             ym = Cac - np.sqrt(Cac**2 + (Rac**2 - a**2)/a**2)
#             print(ym)
#             if np.allclose(b**2 + (y*a)**2 - 2*b*(y*a)*Cbc, Rbc**2):
#                 return y
#             else:
#                 return ym
#         else:
#             return num / den
    
#     ys = [compute_y(x, a, b) for x, a, b in zip(xs, as_, bs)] 
#     print(ys)
# #     print(ys)
    
#     cs = [y*a for y, a, b in zip(ys, as_, bs) if np.allclose(b**2 + (y*a)**2 - 2*b*(y*a)*Cbc, Rbc**2)]
    
#     ret = np.array([(x, a, b, y, c) for (x,a,b,y,c) in zip(xs, as_, bs, ys, cs)])
    
    return ret

In [28]:
p3p(obj_points, img_points, camera_matrix, dist_coeffs)

0.972972972972973 0.9557790087219501 0.5567848665163357


True

In [29]:
np.roots((-0.5625, 3.515625, -5.90625, 3.515625, -0.5625))

array([4.  +0.j        , 1.  +0.00000005j, 1.  -0.00000005j,
       0.25+0.j        ])

In [34]:
tvec + unit_vector(tvec-x)*lengths[0] 

NameError: name 'x' is not defined

In [35]:
unit_vector(tvec - A)

NameError: name 'A' is not defined

In [36]:
unit_vector(z)

NameError: name 'z' is not defined

In [37]:
np.linalg.norm(tvec - A)

NameError: name 'A' is not defined

In [53]:
a, b, c = [np.linalg.norm(tvec - p) for p in obj_points[:3]]
print(a)

2.03100960115899


In [56]:
Rab

0.5

In [39]:
A, B, C = obj_points.reshape(-1,3)[:3]
Rab = np.linalg.norm(B-A)
Rac = np.linalg.norm(C-A)
Rbc = np.linalg.norm(C-B)

In [40]:
Cab = (Rab**2 - a**2 - b**2) / (-2 * a * b) 
Cac = (Rac**2 - a**2 - c**2) / (-2 * a * c) 
Cbc = (Rbc**2 - b**2 - c**2) / (-2 * b * c)
np.array([Cab, Cac, Cbc])

array([0.96969697, 0.93939394, 0.96969697])

In [41]:
cosines

NameError: name 'cosines' is not defined

In [42]:
print(Cab, Cac, Cbc)

0.9696969696969697 0.9393939393939394 0.9696969696969697


In [43]:
xs = p3p_coeffs(Rab, Rac, Rbc, Cab, Cac, Cbc)
as_ = Rab / np.sqrt(xs**2 - 2*xs*Cab + 1)
as_

array([1.83129261, 2.0310096 , 2.04560087, 2.0310096 ])

In [44]:
p, q, r = np.hstack((cv.undistortPoints(img_points, camera_matrix, dist_coeffs).reshape(-1,2)[:3], np.ones((3,1))))

In [45]:
obj_points

array([[0.25, 0.25, 0.  ],
       [0.25, 0.75, 0.  ],
       [0.75, 0.75, 0.  ],
       [0.75, 0.25, 0.  ]])

In [46]:
fx = fy = 750
cx = 640
cy = 512
inv_fx = 1. / fx
inv_fy = 1. / fy
cx_fx = cx / fx
cy_fy = cy / fy

In [47]:
img_points[0]

array([[921.25, 605.75]])

In [57]:
ip = img_points.reshape(-1,2)

mu0, mv0 = ip[0]
mu0 = inv_fx * mu0 - cx_fx;
mv0 = inv_fy * mv0 - cy_fy;
norm = np.sqrt(mu0 * mu0 + mv0 * mv0 + 1);
mk0 = 1. / norm; mu0 *= mk0; mv0 *= mk0;

mu1, mv1 = ip[1]
mu1 = inv_fx * mu1 - cx_fx;
mv1 = inv_fy * mv1 - cy_fy;
norm = np.sqrt(mu1 * mu1 + mv1 * mv1 + 1);
mk1 = 1. / norm; mu1 *= mk1; mv1 *= mk1;

mu2, mv2 = ip[2]
mu2 = inv_fx * mu2 - cx_fx;
mv2 = inv_fy * mv2 - cy_fy;
norm = np.sqrt(mu2 * mu2 + mv2 * mv2 + 1);
mk2 = 1. / norm; mu2 *= mk2; mv2 *= mk2;

In [58]:
cosines = np.zeros(3)
cosines[0] = mu1 * mu2 + mv1 * mv2 + mk1 * mk2;
cosines[1] = mu0 * mu2 + mv0 * mv2 + mk0 * mk2;
cosines[2] = mu0 * mu1 + mv0 * mv1 + mk0 * mk1;
cosines
# Cab, Cac, Cbc = cosines

array([0.98028616, 0.95577901, 0.97297297])

In [59]:
xs = p3p_coeffs(Rab, Rac, Rbc, Cab, Cac, Cbc)
as_ = Rab / np.sqrt(xs**2 - 2*xs*Cab + 1)
as_

array([1.83129261, 2.0310096 , 2.04560087, 2.0310096 ])

In [64]:
np.linalg.norm(tvec - obj_points, axis=1)

array([2.0310096, 2.0310096, 2.0310096, 2.0310096])

In [ ]:
np.array([mu0, mv0, mk0])

In [ ]:
unit_vector(p)

In [ ]:
np.inner(unit_vector(p), unit_vector(q))